# BTW 2025 Data Science Challenge

**TODO Introduction**

## Utils


In [4]:

# TODO Load Data
# TODO Base Model Class
# TODO All Models

## Gathering Domain Knowledge 

**TODO What does the reader have to know about the energy market**

## Data Sources

## Data Cleaning


## Data Analysis

### Comparing Feature Importances

#### AutoGluon

#### Temporal-Fusion-Transformer

## Visualization & Story Telling

### Baseline Models Benchmark

### LSTMs

### Chronos

Chronos is a framework for pre-trained probabilistic time series models introduced by TODO Chronos in March 2024. It tokenizes time series values into a fixed vocabulary through scaling and quantization and trains transformer-based language models on these tokens using the cross-entropy loss function. This means that from a time series after a mean scaling values are taken from defined points of a time series. These context tokens are then used for the (pre)training. Chronos is designed without time-series-specific architecture, resulting in a minimalistic yet effective approach. The framework achieved remarkable results in in-domain experiments and demonstrated competitive zero-shot performance, comparable to models specifically trained on similar tasks.

The developer of Chronos provided a GitHub Repository (https://github.com/amazon-science/chronos-forecasting/tree/main) which enables the user to either use the pretrained models for forecasting fine tune models on their own Data. Compared to the T5 architecture Chronos reduces the vocabulary size resulting in five different models ranging from eight million to 710 million parameters.

In this work, the pretrained Chronos-T5 (Tiny) model was utilized as a benchmark to assess and improve its performance by fine-tuning it on our own data. Specifically, we focused on fine tune the model with the day-ahead electricity prices from ENTSO-E as a domain-specific dataset. The tiny model was chosen for its practicality, as it can be fine-tuned and utilized for forecasting tasks even on a standard laptop. To gain deeper insights into the impact of the dataset size and training steps on model performance, we conducted fine-tuning experiments in four distinct ways.

The energy market has experienced heightened volatility in recent years, driven by geopolitical and economic disruptions such as the Ukraine war. To evaluate the impact of dataset characteristics on model performance, we divided the data into two subsets. The first dataset contains day-ahead prices from January 2022 to December 2023, and the second one data spanning from January 2015 to December 2023. The smaller dataset focuses primarily on recent, highly volatile market conditions, reflecting current dynamics. In contrast, the larger dataset spans a longer historical period, capturing a broader range of market scenarios. This approach enables a direct comparison to determine whether the smaller, more focused dataset enhances adaptability to recent volatility or if the larger dataset provides a more comprehensive foundation due to its diversity.

The developers of Chronos fine-tuned their Chronos-T5 (Small) model with 1000 training steps and achieved remarkable results. We were interested if even more fine-tuning steps could increase the models performance. Therefore, for both datasets, the fine-tuning was executed with two configurations of training steps—1,000 and 10,000 steps. This comprehensive setup aimed to explore the potential of domain-specific fine-tuning in enhancing the model's capabilities.

Chronos models are probabilistic models. Therefore an evaluation which of the fine tuned models performs the best should be done on a large test-set. To determine which configuration performs best for the challenge of predicting 24-hour day-ahead energy prices, we conducted forecasts for a full year, from the beginning of December 2023 to the end of November 2024. For each forecasted day, the context data consisted of the most recent 512 hourly day-ahead prices.

The results are computes as following. For each 24 values of one day we calculated the root mean squared error and the absolute error. For each day the mean of these values are calculated and over the whole year the mean is created again. The yearly Results of the Chronos-T5 (Tiny) model and its fine-tuned versions are presented in the table below.

|Chronos-T5 (Tiny)    |1. Zero-Shot  |2. Fine-Tuned Data: 2015 Steps: 1000|3. Fine-Tuned Data: 2015 Steps: 1000|4. Fine-Tuned Data: 2015 Steps: 1000|5. Fine-Tuned Data: 2015 Steps: 1000|
|------------|------------|------------|------------|------------|------------|
| RMSE              | 25.817  | 22.739    | 23.501 | 24.552 | 26.413 |
| RMSE in percent   | 117.48% | 120.92%   | 146.55% | 128.25% | 137.28% |
| MAE               | 20.127  | 17.254    | 17.969 | 18.59 | 20.263 |
| MAE in percent    | 2637.80% | 2484.01% | 3311.03% | 3168.22% | 4038.85% |

These results reveal several key insights. First, not all fine-tuned models outperform the benchmark. Additionally, models fine-tuned on the larger dataset, which includes data from 2015, tend to perform better than those fine-tuned on the smaller, more recent dataset. This may be due to the fact that, although the energy market is currently highly volatile, less volatile days dominate the market, making the larger dataset, with its broader range of scenarios, more beneficial. Furthermore, the models trained with 1,000 steps, rather than 10,000, tend to perform better. This could indicate that overfitting occurred with the longer training duration.

It is clear that the model which was fine-tuned on the large dataset with 1000 training steps Performs the best. This model has the lowest RMSE and MAE. On average the difference between the predicted hourly day-ahead price and its actual value is 17.254 Euro.

Interesting is that model three and four have still a better performance in the MAE and RMSE, but its corresponding percentage value are outperformed by the benchmark. This shows that with the pretraining the overall error is reduced, but the models are getting worse in predicting extreme scenarios like energy prices above 500 Euros and prices below zero euros.

Model five is outperformed by the benchmark in all metrics.

As already mentioned there are several sizes of Chronos models available. Additional to the Chronos-T5 (Tiny) model we investigated the performance of the Chronos-T5 (Large) model. For this model we performed the same fine-tuning steps as we did for the hronos-T5 (Tiny) model.

The yearly Results of the Chronos-T5 (Large) model and its fine-tuned versions are presented in the table below.

|Chronos-T5 (Large)    |Zero-Shot  |Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|Fine-Tuned Data: 2015 Steps: 1000|
|------------|------------|------------|------------|------------|------------|
| RMSE | 25.803 | 22.818 | Row 1 Col4 | Row 1 Col5 | Row 1 Col6 |
| MAE | 2575.93% | 2604.36% | Row 2 Col4 | Row 2 Col5 | Row 2 Col6 |
| MAE in percent | 2575.93% | 2604.36% | Row 2 Col4 | Row 2 Col5 | Row 2 Col6 |

### Temporal Fusion Transformer

### AutoGluon

### Hacking AutoGluon

## Predictive Modeling

### Final Benchmark

### Final Forecast

In [ ]:
# TODO Download Up to date Data
# TODO Load best model from disk
# TODO Compute Forecast
# TODO Write to File

## Summary

## Future Work

To further improve Chronos one option would be to do a hyperparameter optimization. This would help to find the best hyperparameter configuration for a specific model. 

## Conclusion 